## Tugas:
### Buatlah sebuah model K-Means dengan ketentuan,

1.Gunakan data 'Mall_Customers.csv'

2.Tentukan fitur apa yang tepat untuk melakukan clustering (minimal 2)

3.Buatlah model K-Means dengan mempertimbangkan jumlah k yang terbaik.

4.Buatlah implementasi model SOM untuk segmentasi citra lain

In [1]:
# Gunakan data 'Mall_Customers.csv'
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

df = pd.read_csv('data/Mall_Customers.csv')